In [9]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def read_data():
    filename = 'opx-line.csv'
    df = pd.read_csv(filename, encoding='utf-8', sep=',')
    df = df.dropna(subset=['Unit'])
    df = df.reset_index(drop=True)
    samples = list(set(df['Sample']))
    crystals = list(set(df['Crystal']))
    nums = list(set(df['No']))
    return df, samples, crystals, nums

def calc_formula(df):
    molsi = df['SiO2'] / 60.0843
    molti = df['TiO2'] / 79.8788
    molal = df['Al2O3'] / 101.9613
    molfe = df['FeO'] / 71.8464
    molmn = df['MnO'] / 70.93745
    molmg = df['MgO'] / 40.304
    molca = df['CaO'] / 56.077
    molna = df['Na2O'] / 61.9789
    molk  = df['K2O'] / 94.195
    molcr = df['Cr2O3'] / 151.9904
    molni = df['NiO'] / 74.6894    
    coef = 6 / ( 2 * molsi + 2 * molti + 3 * molal + molfe + molmn + molmg + molca + molna + molk + 3 * molcr + molni )
    # coef = 8 / ( 2 * molsi + 3 * molal + molfe + molmg + molca + molna + molk )
    
    si = molsi * coef
    ti = molti * coef
    al = 2 * molal * coef
    fe = molfe * coef
    mn = molmn * coef
    mg = molmg * coef
    ca = molca * coef
    na = 2 * molna * coef
    k  = 2 * molk  * coef
    cr = 2 * molcr * coef
    ni = molni * coef
    
    cation = si + ti + al + fe + mn + mg + ca + na + k + cr + ni
    # cation = si + al + fe + mg + ca + na + k
    return cation
    
    
def get_data(df, cation, sample, crystal, num):    
    #df = df[ (cation >= 3.98) & (cation <= 4.02) & (df['Total'] >= 97) & (df['Total'] <= 103) ]
    x = df['Distance from rim']
    mgnum = ( 100 * ( df['MgO'] / 40.3044 ) / ( ( df['MgO'] / 40.3044 ) + ( df['FeO'] / 71.844 ) ) ).values
    wo = ( 100 * ( df['CaO'] / 56.0774 ) / ( ( df['MgO'] / 40.3044 ) + ( df['FeO'] / 71.844 ) + ( df['CaO'] / 56.0774 ) ) ).values
    alminum, chromium, titan, manganese = df['Al2O3'].values, df['Cr2O3'].values, df['TiO2'].values, df['MnO'].values
    crystal_name = sample + '_' + crystal + '_' + str(int(num))
    return x, mgnum, wo, alminum, chromium, titan, manganese, crystal_name


def plot_zoning(x, mgnum, wo, alminum, chromium, titan, manganese, crystal_name):
    plt.style.use('default')
    plt.rcParams["font.size"] = 16
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'
    fig = plt.figure( figsize=(4.0, 10.0) )

    ax1 = fig.add_subplot(311)
    ax2 = ax1.twinx()
    ax1.plot(x, mgnum, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='k', c='k')
    ax1.set_ylabel('Mg#')
    ax1.set_ylim(55, 80)
    
    ax2.plot(x, alminum, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='crimson', c='crimson')
    ax2.set_ylim(0.5, 4)
    ax2.spines['right'].set_color('crimson')
    ax2.tick_params(axis='y', colors='crimson')
    ax2.set_ylabel('Al$_2$O$_3$ [wt.%]', color='crimson')
    
    ax1 = fig.add_subplot(312)
    ax2 = ax1.twinx()
    ax1.plot(x, chromium, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='k', c='k')
    ax1.set_ylabel('Cr$_2$O$_2$ [wt.%]')
    ax1.set_ylim(-0.015, 0.4)    
    
    ax2.plot(x, titan, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='crimson', c='crimson')    
    ax2.set_ylim(0.1, 0.5)
    ax2.spines['right'].set_color('crimson')
    ax2.tick_params(axis='y', colors='crimson')
    ax2.set_ylabel('TiO$_2$ [wt.%]', c='crimson')
    
    ax1 = fig.add_subplot(313)
    ax2 = ax1.twinx()
    ax1.plot(x, wo, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='k', c='k')
    ax1.set_ylabel('Wo [mol.%]')
    ax1.set_ylim(2, 7)
    
    ax2.plot(x, manganese, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='crimson', c='crimson')    
    ax2.set_ylim(0.2, 0.7)
    ax2.spines['right'].set_color('crimson')
    ax2.tick_params(axis='y', colors='crimson')
    ax2.set_ylabel('MnO [wt.%]', c='crimson')
    ax1.set_xlabel('Distance from rim [$\mu$m]')

    plt.savefig('opx-line/'+crystal_name+'.jpg', dpi=300, bbox_inches="tight")
    plt.close()
    

def main():
    df, samples, crystals, nums = read_data()
    print(samples, crystals, nums)
    for sample in samples:
        for crystal in crystals:
            for num in nums:
                data = df[ (df['Sample'] == sample) & (df['Crystal'] == crystal) & (df['No'] == num) ]
                
                if len(data) < 1:
                    continue
                else:
                    cation = calc_formula(data)
                    x, mgnum, wo, alminum, chromium, titan, manganese, crystal_name = get_data(data, cation, sample, crystal, num)
                    plot_zoning(x, mgnum, wo, alminum, chromium, titan, manganese, crystal_name)
    
    
if __name__ == "__main__":
    main()

['Okama 4-1-1', 'Okama 16082606-1', 'Okama 4-4-3', 'Okama 16081606-1', 'Okama 5-2-1', 'Okama 4-2-3', 'Okama 4-3-1'] ['px-12', 'px-23', 'px-01', 'px-03', 'px-29', 'px-16', 'px-04', 'px-13', 'px-11', 'px-30', 'px-32', 'px-18', 'px-31', 'px-25', 'px-21', 'px-02', 'px-09', 'px-08', 'px-14', 'px-05', 'px-07', 'px-06'] [1.0, 2.0, 3.0, 4.0]


In [6]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def read_data():
    filename = 'cpx-line.csv'
    df = pd.read_csv(filename, encoding='utf-8', sep=',')
    df = df.dropna(subset=['Unit'])
    df = df.reset_index(drop=True)
    samples = list(set(df['Sample']))
    crystals = list(set(df['Crystal']))
    nums = list(set(df['No']))
    return df, samples, crystals, nums

def calc_formula(df):
    molsi = df['SiO2'] / 60.0843
    molti = df['TiO2'] / 79.8788
    molal = df['Al2O3'] / 101.9613
    molfe = df['FeO'] / 71.8464
    molmn = df['MnO'] / 70.93745
    molmg = df['MgO'] / 40.304
    molca = df['CaO'] / 56.077
    molna = df['Na2O'] / 61.9789
    molk  = df['K2O'] / 94.195
    molcr = df['Cr2O3'] / 151.9904
    molni = df['NiO'] / 74.6894    
    coef = 6 / ( 2 * molsi + 2 * molti + 3 * molal + molfe + molmn + molmg + molca + molna + molk + 3 * molcr + molni )
    # coef = 8 / ( 2 * molsi + 3 * molal + molfe + molmg + molca + molna + molk )
    
    si = molsi * coef
    ti = molti * coef
    al = 2 * molal * coef
    fe = molfe * coef
    mn = molmn * coef
    mg = molmg * coef
    ca = molca * coef
    na = 2 * molna * coef
    k  = 2 * molk  * coef
    cr = 2 * molcr * coef
    ni = molni * coef
    
    cation = si + ti + al + fe + mn + mg + ca + na + k + cr + ni
    # cation = si + al + fe + mg + ca + na + k
    return cation
    
    
def get_data(df, cation, sample, crystal, num):    
    df = df[ (cation >= 3.97) & (cation <= 4.03) & (df['Total'] >= 95) & (df['Total'] <= 103) ]
    x = df['Distance from rim']
    mgnum = ( 100 * ( df['MgO'] / 40.3044 ) / ( ( df['MgO'] / 40.3044 ) + ( df['FeO'] / 71.844 ) ) ).values
    wo = ( 100 * ( df['CaO'] / 56.0774 ) / ( ( df['MgO'] / 40.3044 ) + ( df['FeO'] / 71.844 ) + ( df['CaO'] / 56.0774 ) ) ).values
    alminum, chromium, titan, manganese = df['Al2O3'].values, df['Cr2O3'].values, df['TiO2'].values, df['MnO'].values
    crystal_name = sample + '_' + crystal + '_' + str(int(num))
    return x, mgnum, wo, alminum, chromium, titan, manganese, crystal_name


def plot_zoning(x, mgnum, wo, alminum, chromium, titan, manganese, crystal_name):
    plt.style.use('default')
    plt.rcParams["font.size"] = 16
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'
    fig = plt.figure( figsize=(4.0, 10.0) )

    ax1 = fig.add_subplot(311)
    ax2 = ax1.twinx()
    ax1.plot(x, mgnum, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='k', c='k')
    ax1.set_ylabel('Mg#')
    ax1.set_ylim(55, 80)
    
    ax2.plot(x, alminum, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='crimson', c='crimson')
    ax2.set_ylim(0.5, 4)
    ax2.spines['right'].set_color('crimson')
    ax2.tick_params(axis='y', colors='crimson')
    ax2.set_ylabel('Al$_2$O$_3$ [wt.%]', color='crimson')
    
    ax1 = fig.add_subplot(312)
    ax2 = ax1.twinx()
    ax1.plot(x, chromium, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='k', c='k')
    ax1.set_ylabel('Cr$_2$O$_2$ [wt.%]')
    ax1.set_ylim(-0.015, 0.4)    
    
    ax2.plot(x, titan, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='crimson', c='crimson')    
    ax2.set_ylim(0.3, 1)
    ax2.spines['right'].set_color('crimson')
    ax2.tick_params(axis='y', colors='crimson')
    ax2.set_ylabel('TiO$_2$ [wt.%]', c='crimson')
    
    ax1 = fig.add_subplot(313)
    ax2 = ax1.twinx()
    ax1.plot(x, wo, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='k', c='k')
    ax1.set_ylabel('Wo [mol.%]')
    ax1.set_ylim(32, 42)
    
    ax2.plot(x, manganese, '--o', ms=3, mew=0.5, mfc='w', lw=0.5, mec='crimson', c='crimson')    
    ax2.set_ylim(0.2, 0.7)
    ax2.spines['right'].set_color('crimson')
    ax2.tick_params(axis='y', colors='crimson')
    ax2.set_ylabel('MnO [wt.%]', c='crimson')
    ax1.set_xlabel('Distance from rim [$\mu$m]')

    plt.savefig('cpx-line/'+crystal_name+'.jpg', dpi=300, bbox_inches="tight")
    plt.close()
    

def main():
    df, samples, crystals, nums = read_data()
    print(samples, crystals, nums)
    for sample in samples:
        for crystal in crystals:
            for num in nums:
                data = df[ (df['Sample'] == sample) & (df['Crystal'] == crystal) & (df['No'] == num) ]
                
                if len(data) < 1:
                    continue
                else:
                    cation = calc_formula(data)
                    x, mgnum, wo, alminum, chromium, titan, manganese, crystal_name = get_data(data, cation, sample, crystal, num)
                    plot_zoning(x, mgnum, wo, alminum, chromium, titan, manganese, crystal_name)
    
    
if __name__ == "__main__":
    main()

['Okama 4-1-1', 'Okama 5-2-1', 'Okama 4-4-3', 'Okama 4-3-1'] ['px-01', 'px-02', 'px-17', 'px-04'] [1.0, 2.0, 3.0]
